In [1]:
	
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, Input

In [2]:
# Training datasets
cgm_train = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/Train/cgm_train.csv')
image_train = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/Train/img_train.csv')
demo_viome_train = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/Train/demo_viome_train.csv')
label_train = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/Train/label_train.csv')

# Test datasets
cgm_test = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/cgm_test.csv')
image_test = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/img_test.csv')
demo_viome_test = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/demo_viome_test.csv')
label_test = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/label_test_breakfast_only.csv')

In [3]:
# Merge CGM and Image datasets
temp_data = pd.merge(image_train, cgm_train, on=['Subject ID', 'Day'])
output_labels = label_train[["Subject ID", "Day", "Lunch Calories"]]
data_train = pd.merge(temp_data, output_labels, on=['Subject ID','Day'])

In [4]:
data_train

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories
0,1,2,"[[[140, 122, 108], [135, 118, 104], [118, 104,...","[[[41, 152, 201], [77, 164, 205], [88, 157, 13...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (...",830
1,1,3,"[[[67, 58, 47], [59, 52, 41], [51, 45, 35], [4...","[[[40, 59, 77], [35, 56, 72], [20, 36, 47], [9...",2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (...",435
2,1,4,"[[[199, 195, 193], [198, 193, 192], [196, 192,...","[[[53, 44, 38], [51, 43, 36], [54, 47, 39], [4...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ...",555
3,1,5,"[[[149, 121, 80], [157, 128, 86], [159, 130, 8...","[[[30, 28, 28], [20, 18, 17], [31, 27, 23], [2...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ...",355
4,1,6,"[[[175, 184, 198], [192, 206, 219], [160, 165,...","[[[74, 85, 100], [59, 69, 81], [73, 84, 96], [...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ...",1180
...,...,...,...,...,...,...,...,...
319,7,6,"[[[68, 34, 35], [82, 60, 51], [63, 55, 38], [3...","[[[90, 77, 75], [92, 78, 75], [94, 83, 81], [9...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[('2021-12-18 08:50:00', 101.36), ('2021-12-18...",1180
320,7,7,"[[[26, 26, 22], [17, 17, 13], [18, 19, 14], [9...","[[[17, 9, 8], [10, 7, 7], [3, 3, 4], [3, 3, 3]...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[('2021-12-19 08:40:00', 100.68), ('2021-12-19...",830
321,7,8,"[[[43, 37, 33], [42, 36, 31], [42, 37, 33], [4...","[[[122, 108, 107], [124, 110, 108], [124, 111,...",2021-12-20 09:06:00,2021-12-20 12:46:00,"[('2021-12-20 09:00:00', 104.04), ('2021-12-20...",435
322,7,9,"[[[41, 38, 33], [41, 38, 33], [41, 38, 33], [4...","[[[59, 46, 32], [63, 51, 41], [57, 42, 28], [6...",2021-12-21 08:34:00,2021-12-21 12:38:00,"[('2021-12-21 08:25:00', 96.4), ('2021-12-21 0...",555


In [5]:
def create_placeholder_image(size=(64, 64, 3)):
    return np.zeros(size, dtype=np.float32)  # Normalized [0, 1] range

# Function to preprocess image data
def handle_image(img_data, size=(64, 64)):
    try:
        img_array = np.array(img_data, dtype=np.uint8)  # Ensure valid data type

        # Check for empty image
        if img_array.size == 0 or img_array.ndim != 3 or img_array.shape[2] != 3:
            raise ValueError(f"Invalid or empty image dimensions: {img_array.shape}")

        img_resized = np.array(Image.fromarray(img_array).resize(size))  # Resize
        img_normalized = img_resized / 255.0  # Normalize pixel values to [0, 1]
        return img_normalized
    except Exception as e:
        print(f"Error preprocessing image: {e}")
        return create_placeholder_image(size)

# Preprocess the dataset
def preprocess_img(data):
    # Define placeholder image
    placeholder_image = create_placeholder_image()

    # Iterate over rows to preprocess images
    breakfast_images = []
    lunch_images = []

    for index, row in data.iterrows():
        # Handle missing breakfast images
        if pd.isnull(row['Image Before Breakfast']) or row['Image Before Breakfast'] == '[]':  # Check for empty list or NaN
            breakfast_images.append(placeholder_image)
        else:
            try:
                img_data = eval(row['Image Before Breakfast'])  # Convert string to list
                breakfast_images.append(handle_image(img_data))
            except Exception as e:
                print(f"Error at index {index}, breakfast: {e}")
                breakfast_images.append(placeholder_image)

        # Handle missing lunch images
        if pd.isnull(row['Image Before Lunch']) or row['Image Before Lunch'] == '[]':  # Check for empty list or NaN
            lunch_images.append(placeholder_image)
        else:
            try:
                img_data = eval(row['Image Before Lunch'])  # Convert string to list
                lunch_images.append(handle_image(img_data))
            except Exception as e:
                print(f"Error at index {index}, lunch: {e}")
                lunch_images.append(placeholder_image)

    # Add preprocessed images back to the dataset
    data['Image Before Breakfast'] = breakfast_images
    data['Image Before Lunch'] = lunch_images

    return data

In [6]:
data_train = preprocess_img(data_train)
data_train

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories
0,1,2,"[[[0.5490196078431373, 0.47843137254901963, 0....","[[[0.1607843137254902, 0.596078431372549, 0.78...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (...",830
1,1,3,"[[[0.2627450980392157, 0.22745098039215686, 0....","[[[0.1568627450980392, 0.23137254901960785, 0....",2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (...",435
2,1,4,"[[[0.7803921568627451, 0.7647058823529411, 0.7...","[[[0.20784313725490197, 0.17254901960784313, 0...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ...",555
3,1,5,"[[[0.5843137254901961, 0.4745098039215686, 0.3...","[[[0.11764705882352941, 0.10980392156862745, 0...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ...",355
4,1,6,"[[[0.6862745098039216, 0.7215686274509804, 0.7...","[[[0.2901960784313726, 0.3333333333333333, 0.3...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ...",1180
...,...,...,...,...,...,...,...,...
319,7,6,"[[[0.26666666666666666, 0.13333333333333333, 0...","[[[0.35294117647058826, 0.30196078431372547, 0...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[('2021-12-18 08:50:00', 101.36), ('2021-12-18...",1180
320,7,7,"[[[0.10196078431372549, 0.10196078431372549, 0...","[[[0.06666666666666667, 0.03529411764705882, 0...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[('2021-12-19 08:40:00', 100.68), ('2021-12-19...",830
321,7,8,"[[[0.16862745098039217, 0.1450980392156863, 0....","[[[0.47843137254901963, 0.4235294117647059, 0....",2021-12-20 09:06:00,2021-12-20 12:46:00,"[('2021-12-20 09:00:00', 104.04), ('2021-12-20...",435
322,7,9,"[[[0.1607843137254902, 0.14901960784313725, 0....","[[[0.23137254901960785, 0.1803921568627451, 0....",2021-12-21 08:34:00,2021-12-21 12:38:00,"[('2021-12-21 08:25:00', 96.4), ('2021-12-21 0...",555


In [7]:
# Updating missing values with mean
def preprocess_cgm(data_train):

    # Function to check if CGM Data is an empty array
    def is_cgm_data_empty(row):
        try:
            cgm_list = ast.literal_eval(row['CGM Data'])
            return len(cgm_list) == 0
        except:
            return True

    # Function to filter out rows where CGM Data is empty
    data_train = data_train[~data_train.apply(is_cgm_data_empty, axis=1)]

    # Get the HH:MM:SS breakfast and lunch times and calculate the mean for each subject ID
    data_train.loc[:, 'Breakfast Time'] = pd.to_datetime(data_train['Breakfast Time'], errors='coerce')
    data_train.loc[:, 'Lunch Time'] = pd.to_datetime(data_train['Lunch Time'], errors='coerce')

    def mean_time(times):
        total_seconds = sum([t.hour * 3600 + t.minute * 60 + t.second for t in times if pd.notna(t)])
        mean_seconds = total_seconds // len([t for t in times if pd.notna(t)])
        return pd.to_datetime(mean_seconds, unit='s').time()

    mean_times = data_train.groupby('Subject ID')[['Breakfast Time', 'Lunch Time']].apply(
        lambda group: pd.Series({
            'Breakfast Time': mean_time(group['Breakfast Time']),
            'Lunch Time': mean_time(group['Lunch Time'])
        })
    )

    mean_times = mean_times.reset_index()

    # Find the reference date for any row within the same subject:
    def get_reference_date(subject_id):
        day_2_breakfast_index = data_train[(data_train['Subject ID'] == subject_id) & (data_train['Day'] == 2)]['Breakfast Time'].first_valid_index()
        if day_2_breakfast_index is None or pd.isna(data_train.loc[day_2_breakfast_index, 'Breakfast Time']):
            # If Breakfast Time is not available for Day 2, check Lunch Time
            day_2_lunch_index = data_train[(data_train['Subject ID'] == subject_id) & (data_train['Day'] == 2)]['Lunch Time'].first_valid_index()
            if day_2_lunch_index is not None:
                reference_date = data_train.loc[day_2_lunch_index, 'Lunch Time']
                reference_day = 2
            else:
                # If neither Breakfast nor Lunch time is available for Day 2, check Day 3
                day_3_breakfast_index = data_train[(data_train['Subject ID'] == subject_id) & (data_train['Day'] == 3)]['Breakfast Time'].first_valid_index()
                if day_3_breakfast_index is not None:
                    reference_date = data_train.loc[day_3_breakfast_index, 'Breakfast Time']
                    reference_day = 3
                else:
                    reference_date = None
                    reference_day = None
        else:
            reference_date = data_train.loc[day_2_breakfast_index, 'Breakfast Time']
            reference_day = 2
        
        return pd.Series([reference_date.date(), reference_day])

    mean_times[['Reference Date', 'Reference Day']] = mean_times['Subject ID'].apply(get_reference_date)

    # Update missing values
    def update_missing_breakfast_time(row, mean_times):
        subject_id = row['Subject ID']
        day = row['Day']
        
        mean_breakfast_time = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Breakfast Time'].iloc[0]
        mean_reference_date = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Date'].iloc[0]
        reference_day = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Day'].iloc[0]
        current_date = mean_reference_date + pd.Timedelta(days=(day - reference_day))
        updated_breakfast_time = pd.to_datetime(current_date.strftime('%Y-%m-%d') + ' ' + mean_breakfast_time.strftime('%H:%M:%S'))
        row['Breakfast Time'] = updated_breakfast_time
        return row

    data_train = data_train.apply(
        lambda row: update_missing_breakfast_time(row, mean_times) if pd.isna(row['Breakfast Time']) else row, axis=1
    )

    def update_missing_lunch_time(row, mean_times):
        subject_id = row['Subject ID']
        day = row['Day']
        mean_lunch_time = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Lunch Time'].iloc[0]
        mean_reference_date = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Date'].iloc[0]
        reference_day = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Day'].iloc[0]
        current_date = mean_reference_date + pd.Timedelta(days=(day - reference_day))
        updated_lunch_time = pd.to_datetime(current_date.strftime('%Y-%m-%d') + ' ' + mean_lunch_time.strftime('%H:%M:%S'))
        row['Lunch Time'] = updated_lunch_time
        return row

    data_train = data_train.apply(
        lambda row: update_missing_lunch_time(row, mean_times) if pd.isna(row['Lunch Time']) else row, axis=1
    )

    data_train['Time Between Meals'] = (data_train['Lunch Time'] - data_train['Breakfast Time']).dt.total_seconds()

    return data_train

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Function to calculate MAGE for a given glucose series
def calculate_mage(glucose_data):
    # Calculate the mean glucose level for the period
    mean_glucose = glucose_data.mean()
    
    # Calculate the glucose excursions (absolute differences from mean)
    excursions = np.abs(glucose_data - mean_glucose)
    
    # Calculate the mean amplitude of glucose excursions (MAGE)
    mage = excursions.mean()
    
    return mage

# Function to parse the CGM data from string format in the CSV
def parse_cgm_data(cgm_str):
    # Convert the string of tuples to an actual list of tuples
    cgm_data = eval(cgm_str)  # Be cautious with eval; here it assumes safe input
    return [(datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S'), glucose) for timestamp, glucose in cgm_data]

# Function to detect meal times based on glucose excursions
def detect_meal_times(df, min_mage=20, meal_window_start='06:00', meal_window_end='14:00'):
    meal_times = []

    # Loop through each day (assuming data is continuous and covers multiple days)
    for _, row in df.iterrows():
        # Extract the date, meal times, and CGM data
        meal_times_for_day = {'breakfast': row['Breakfast Time'], 'lunch': row['Lunch Time']}
        cgm_data = row['CGM Data']
        cgm_df = pd.DataFrame(cgm_data, columns=['timestamp', 'glucose'])
        cgm_df['timestamp'] = pd.to_datetime(cgm_df['timestamp'])
        cgm_df.set_index('timestamp', inplace=True)
        
        # Calculate MAGE for the day
        mage = calculate_mage(cgm_df['glucose'])
        print(f"MAGE for Subject {row['Subject ID']} on {row['Day']}: {mage}")

        if mage > min_mage:
            # Detect glucose peaks during meal windows
            breakfast_window = cgm_df.between_time('06:00', '09:00')
            lunch_window = cgm_df.between_time('11:00', '14:00')

            breakfast_peak_time = breakfast_window['glucose'].idxmax() if not breakfast_window.empty else None
            lunch_peak_time = lunch_window['glucose'].idxmax() if not lunch_window.empty else None

            if breakfast_peak_time:
                meal_times.append(('Breakfast', breakfast_peak_time))
            if lunch_peak_time:
                meal_times.append(('Lunch', lunch_peak_time))

    return meal_times

# Function to impute missing meal times
def impute_missing_meals(df, meal_times):
    # Loop through meal_times and assign the detected times to the DataFrame where values are missing
    for meal, peak_time in meal_times:
        for idx, row in df.iterrows():
            # Check if 'Breakfast Time' is '{}' and impute the peak time
            if meal == 'Breakfast' and row['Breakfast Time'] == '{}':
                df.at[idx, 'Breakfast Time'] = peak_time
            if meal == 'Lunch' and row['Lunch Time'] == '{}':
                df.at[idx, 'Lunch Time'] = peak_time

    return df

# Read the CSV data
num_missing = data_train[data_train['Breakfast Time'] == '{}'].shape[0]
print(num_missing)
num_missing_1 = data_train[data_train['Lunch Time'] == '{}'].shape[0]
print(num_missing_1)

# Parse the CGM data from the string format in CSV
data_train['CGM Data'] = data_train['CGM Data'].apply(parse_cgm_data)

# Detect meal times based on glucose data
meal_times = detect_meal_times(data_train)
# print(f"Detected meal times: {meal_times}")

# Impute missing meal times based on detected peaks
data_train = impute_missing_meals(data_train, meal_times)
# print(f"Data with imputed meal times:\n{df[['Subject ID', 'Day', 'Breakfast Time', 'Lunch Time']]}")

# Optionally: Plot glucose data and highlight detected meal times (for visualization)
# plt.figure(figsize=(10, 6))

# Plot glucose data for each subject
# for _, row in df.iterrows():
#     cgm_data = row['CGM Data']
#     cgm_df = pd.DataFrame(cgm_data, columns=['timestamp', 'glucose'])
#     cgm_df['timestamp'] = pd.to_datetime(cgm_df['timestamp'])
#     plt.plot(cgm_df['timestamp'], cgm_df['glucose'], label=f"Subject {row['Subject ID']}")

# # Add vertical lines for detected meal times
# for meal, time in meal_times:
#     plt.axvline(x=time, linestyle='--', label=f'{meal} Time')

# plt.legend()
# plt.xlabel('Time')
# plt.ylabel('Glucose Level (mg/dL)')
# plt.title('CGM Glucose Levels with Detected Meal Times')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()




26
24
MAGE for Subject 1 on 2: 15.949690714244408
MAGE for Subject 1 on 3: 4.491486672456695
MAGE for Subject 1 on 4: 10.129922534397041
MAGE for Subject 1 on 5: 8.626956741898148
MAGE for Subject 1 on 6: 4.691067791444557
MAGE for Subject 1 on 7: 7.49072261557684
MAGE for Subject 1 on 8: 8.476056836935959
MAGE for Subject 1 on 9: 8.529128141052338
MAGE for Subject 1 on 10: 6.286510120775994
MAGE for Subject 2 on 2: 24.137382716049384
MAGE for Subject 2 on 3: 21.292734995383192
MAGE for Subject 2 on 4: 37.26590830415007
MAGE for Subject 2 on 5: 13.90038752362949
MAGE for Subject 2 on 6: 29.691567402471186
MAGE for Subject 2 on 7: 40.57486111111111
MAGE for Subject 2 on 8: 11.850834374566155
MAGE for Subject 2 on 9: 31.52696931833958
MAGE for Subject 2 on 10: 15.863679022629459
MAGE for Subject 3 on 2: 37.80515687907816
MAGE for Subject 3 on 3: 16.763170076039398
MAGE for Subject 3 on 4: 40.81753073298615
MAGE for Subject 3 on 5: 20.84662987644037
MAGE for Subject 3 on 6: 15.94550256837

In [9]:
num_missing = data_train[data_train['Breakfast Time'] == '{}'].shape[0]
print(num_missing)
num_missing_1 = data_train[data_train['Lunch Time'] == '{}'].shape[0]
print(num_missing_1)

0
0


In [10]:
num_missing = data_train[data_train['CGM Data'] == '[]'].shape[0]
print(num_missing)

0


In [11]:
data_train = data_train[data_train['CGM Data'] != '[]']

In [12]:
# data_train = preprocess_cgm(data_train)

# data_train

In [13]:
def preprocess_viome(data):
    # Split the `Viome` column into individual features
    viome_split = data['Viome'].str.split(',', expand=True).astype(float)
    viome_split.columns = [f"Viome_{i}" for i in range(viome_split.shape[1])]
    data = pd.concat([data.drop(columns=['Viome']), viome_split], axis=1)

    # Impute missing values for numeric columns
    numeric_cols = data.select_dtypes(include=[np.number]).columns.drop('Subject ID')
    imputer = SimpleImputer(strategy='mean')
    data[numeric_cols] = imputer.fit_transform(data[numeric_cols])

    # Encode categorical columns
    categorical_cols = ['Race', 'Diabetes Status']
    encoder = OneHotEncoder(sparse_output=False, drop='first') 
    encoded_cats = pd.DataFrame(
        encoder.fit_transform(data[categorical_cols]),
        columns=encoder.get_feature_names_out(categorical_cols)
    )

    # Drop original categorical columns and merge encoded ones
    data = pd.concat([data.drop(columns=categorical_cols), encoded_cats], axis=1)

    return data

In [14]:
demo_viome_train = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/Train/demo_viome_train.csv')
demo_viome_train = preprocess_viome(demo_viome_train)
final_data = pd.merge(data_train, demo_viome_train, on=['Subject ID'])

final_data

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories,Age,Gender,...,Viome_21,Viome_22,Viome_23,Viome_24,Viome_25,Viome_26,Race_Hispanic/Latino,Race_White,Diabetes Status_2.0,Diabetes Status_3.0
0,1,2,"[[[0.5490196078431373, 0.47843137254901963, 0....","[[[0.1607843137254902, 0.596078431372549, 0.78...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[(2021-09-19 08:20:00, 98.26666666666667), (20...",830,27.0,0.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
1,1,3,"[[[0.2627450980392157, 0.22745098039215686, 0....","[[[0.1568627450980392, 0.23137254901960785, 0....",2021-09-20 09:50:00,2021-09-20 15:20:00,"[(2021-09-20 09:10:00, 97.18333333333334), (20...",435,27.0,0.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
2,1,4,"[[[0.7803921568627451, 0.7647058823529411, 0.7...","[[[0.20784313725490197, 0.17254901960784313, 0...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[(2021-09-21 09:20:00, 107.36666666666666), (2...",555,27.0,0.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
3,1,5,"[[[0.5843137254901961, 0.4745098039215686, 0.3...","[[[0.11764705882352941, 0.10980392156862745, 0...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[(2021-09-22 09:25:00, 107.28333333333333), (2...",355,27.0,0.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
4,1,6,"[[[0.6862745098039216, 0.7215686274509804, 0.7...","[[[0.2901960784313726, 0.3333333333333333, 0.3...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[(2021-09-23 08:55:00, 103.0), (2021-09-23 09:...",1180,27.0,0.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,7,6,"[[[0.26666666666666666, 0.13333333333333333, 0...","[[[0.35294117647058826, 0.30196078431372547, 0...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[(2021-12-18 08:50:00, 101.36), (2021-12-18 08...",1180,66.0,1.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0
320,7,7,"[[[0.10196078431372549, 0.10196078431372549, 0...","[[[0.06666666666666667, 0.03529411764705882, 0...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[(2021-12-19 08:40:00, 100.68), (2021-12-19 08...",830,66.0,1.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0
321,7,8,"[[[0.16862745098039217, 0.1450980392156863, 0....","[[[0.47843137254901963, 0.4235294117647059, 0....",2021-12-20 09:06:00,2021-12-20 12:46:00,"[(2021-12-20 09:00:00, 104.04), (2021-12-20 09...",435,66.0,1.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0
322,7,9,"[[[0.1607843137254902, 0.14901960784313725, 0....","[[[0.23137254901960785, 0.1803921568627451, 0....",2021-12-21 08:34:00,2021-12-21 12:38:00,"[(2021-12-21 08:25:00, 96.4), (2021-12-21 08:3...",555,66.0,1.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0


In [15]:
numeric_cols = final_data.select_dtypes(include=[np.number]).columns.difference(['Subject ID', 'Day', 'Lunch Calories'])
scaler = StandardScaler()
final_data[numeric_cols] = scaler.fit_transform(final_data[numeric_cols])

# Perform PCA
pca = PCA()
X_pca = pca.fit_transform(final_data[numeric_cols])

# Select the number of components that explain at least 90% of the variance
n_components = sum(pca.explained_variance_ratio_.cumsum() <= 0.90)
print(f"Number of components to explain 90% variance: {n_components}")

# Apply PCA with the selected number of components
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(final_data[numeric_cols])

# Create a DataFrame with reduced features, preserving the component names
columns = [f"PC{i+1}" for i in range(n_components)]
reduced_df = pd.DataFrame(X_reduced, columns=columns, index=final_data.index)

# Select non-numerical and excluded columns from the original data
excluded_cols = final_data.drop(columns=numeric_cols)

# Combine reduced features with the excluded columns, keeping column names intact
final_data = pd.concat([excluded_cols, reduced_df], axis=1)

Number of components to explain 90% variance: 21


In [16]:
final_data.columns

Index(['Subject ID', 'Day', 'Image Before Breakfast', 'Image Before Lunch',
       'Breakfast Time', 'Lunch Time', 'CGM Data', 'Lunch Calories', 'PC1',
       'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11',
       'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20',
       'PC21'],
      dtype='object')

In [17]:
class MultimodalDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
        # Convert images to tensors and permute dimensions to [batch, channels, height, width]
        self.breakfast_images = torch.tensor(
            np.stack(data['Image Before Breakfast'].values), 
            dtype=torch.float32
        ).permute(0, 3, 1, 2)  # Change from NHWC to NCHW format
        
        self.lunch_images = torch.tensor(
            np.stack(data['Image Before Lunch'].values), 
            dtype=torch.float32
        ).permute(0, 3, 1, 2)  # Change from NHWC to NCHW format
        
        # Process CGM data
        def process_cgm(x):
            try:
                if isinstance(x, str):
                    clean_str = x.replace('[', '').replace(']', '').replace('\n', '').replace(' ', '')
                    if clean_str:
                        values = [float(i) for i in clean_str.split(',') if i]
                        if len(values) > 288:
                            return values[:288]
                        elif len(values) < 288:
                            return values + [0.0] * (288 - len(values))
                        return values
                return [0.0] * 288
            except:
                return [0.0] * 288
        
        processed_cgm = []
        for x in data['CGM Data'].values:
            cgm_values = process_cgm(x)
            processed_cgm.append(cgm_values)
            
        self.cgm_data = torch.tensor(processed_cgm, dtype=torch.float32)
        
        # Extract viome and demographic features
        viome_cols = [col for col in data.columns if 'PC' in col]
        demo_viome_features = data[viome_cols].values

        # demo_cols = ['Age', 'Weight', 'Height', 'A1C', 'Baseline Fasting Glucose', 
        #              'Insulin', 'Triglycerides', 'Cholesterol', 'HDL', 'Non-HDL', 
        #              'LDL', 'VLDL', 'CHO/HDL Ratio', 'HOMA-IR', 'BMI']
        
        # demo_viome_features = data[viome_cols + demo_cols].values
        self.demo_viome = torch.tensor(demo_viome_features, dtype=torch.float32)
        
        # Extract labels
        self.labels = torch.tensor(data['Lunch Calories'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return {
            'breakfast_img': self.breakfast_images[idx],
            'lunch_img': self.lunch_images[idx],
            'cgm': self.cgm_data[idx],
            'demo_viome': self.demo_viome[idx],
            'label': self.labels[idx]
        }

In [18]:
class MultimodalNet(nn.Module):
    def __init__(self, cgm_dim, demo_viome_dim):
        super(MultimodalNet, self).__init__()
        
        # Image encoders (modified for correct dimensions)
        self.image_encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Input: [B, 3, 64, 64]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: [B, 32, 32, 32]
            nn.Conv2d(32, 64, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: [B, 64, 16, 16]
            nn.Flatten(),                                # Output: [B, 64 * 16 * 16]
            nn.Linear(64 * 16 * 16, 256),               # Output: [B, 256]
            nn.ReLU()
        )
        # CGM encoder
        self.cgm_encoder = nn.Sequential(
            nn.Linear(288, 128),  # Assuming CGM length is 288
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        
        # Demo-Viome encoder
        self.demo_viome_encoder = nn.Sequential(
            nn.Linear(demo_viome_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Joint embedding
        self.fusion = nn.Sequential(
            nn.Linear(256 * 2 + 64 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, breakfast_img, lunch_img, cgm, demo_viome):
        # Ensure correct dimensions for images
        if breakfast_img.dim() == 3:
            breakfast_img = breakfast_img.unsqueeze(0)
        if lunch_img.dim() == 3:
            lunch_img = lunch_img.unsqueeze(0)
            
        # Encode images
        breakfast_features = self.image_encoder(breakfast_img)
        lunch_features = self.image_encoder(lunch_img)
        
        # Encode CGM
        cgm_features = self.cgm_encoder(cgm)
        
        # Encode demo-viome
        demo_viome_features = self.demo_viome_encoder(demo_viome)
        
        # Concatenate all features
        combined = torch.cat([
            breakfast_features, 
            lunch_features, 
            cgm_features, 
            demo_viome_features
        ], dim=1)
        
        # Final prediction
        output = self.fusion(combined)
        return output.squeeze()

In [19]:
class RMSRELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super(RMSRELoss, self).__init__()
        self.eps = eps
        
    def forward(self, pred, target):
        relative_error = (pred - target) / (target + self.eps)
        rmsre = torch.sqrt(torch.mean(relative_error ** 2))
        return rmsre

In [20]:
print("Shape of breakfast images:", final_data['Image Before Breakfast'].iloc[0].shape)
print("Number of Viome columns:", len([col for col in final_data.columns if 'PC' in col]))

dataset = MultimodalDataset(final_data)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model
cgm_dim = dataset.cgm_data.shape[1]  # Should be 288
demo_viome_dim = dataset.demo_viome.shape[1] 

model = MultimodalNet(
    cgm_dim=dataset.cgm_data.shape[1],
    demo_viome_dim=dataset.demo_viome.shape[1]
)

# Initialize optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = RMSRELoss()

# Training loop
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    
    for batch in loader:
        optimizer.zero_grad()
        
        pred = model(
            batch['breakfast_img'],
            batch['lunch_img'],
            batch['cgm'],
            batch['demo_viome']
        )
        
        loss = criterion(pred, batch['label'])
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

Shape of breakfast images: (64, 64, 3)
Number of Viome columns: 21


In [21]:
num_epochs = 50
for epoch in range(num_epochs):
    loss = train_epoch(model, train_loader, optimizer, criterion)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}')

Epoch 1, Loss: 0.9139
Epoch 2, Loss: 0.4438
Epoch 3, Loss: 0.3761
Epoch 4, Loss: 0.3730
Epoch 5, Loss: 0.3715
Epoch 6, Loss: 0.3605
Epoch 7, Loss: 0.3464
Epoch 8, Loss: 0.3514
Epoch 9, Loss: 0.3461
Epoch 10, Loss: 0.3446
Epoch 11, Loss: 0.3520
Epoch 12, Loss: 0.3403
Epoch 13, Loss: 0.3491
Epoch 14, Loss: 0.3325
Epoch 15, Loss: 0.3316
Epoch 16, Loss: 0.3519
Epoch 17, Loss: 0.3512
Epoch 18, Loss: 0.3385
Epoch 19, Loss: 0.3459
Epoch 20, Loss: 0.3476
Epoch 21, Loss: 0.3282
Epoch 22, Loss: 0.3617
Epoch 23, Loss: 0.3243
Epoch 24, Loss: 0.3448
Epoch 25, Loss: 0.3519
Epoch 26, Loss: 0.3279
Epoch 27, Loss: 0.3431
Epoch 28, Loss: 0.3287
Epoch 29, Loss: 0.3467
Epoch 30, Loss: 0.3581
Epoch 31, Loss: 0.3206
Epoch 32, Loss: 0.3385
Epoch 33, Loss: 0.3569
Epoch 34, Loss: 0.3430
Epoch 35, Loss: 0.3569
Epoch 36, Loss: 0.3340
Epoch 37, Loss: 0.3320
Epoch 38, Loss: 0.3501
Epoch 39, Loss: 0.3334
Epoch 40, Loss: 0.3401
Epoch 41, Loss: 0.3401
Epoch 42, Loss: 0.3365
Epoch 43, Loss: 0.3629
Epoch 44, Loss: 0.36

In [22]:
# Prepare test data
temp_test_data = pd.merge(image_test, cgm_test, on=['Subject ID', 'Day'])
processed_test_data = preprocess_img(temp_test_data)
# processed_test_data = preprocess_cgm(processed_test_data)
# Parse the CGM data from the string format in CSV
temp_test_data['CGM Data'] = temp_test_data['CGM Data'].apply(parse_cgm_data)

# Detect meal times based on glucose data
meal_times_test = detect_meal_times(temp_test_data)
# print(f"Detected meal times: {meal_times}")

# Impute missing meal times based on detected peaks
temp_test_data = impute_missing_meals(temp_test_data, meal_times)
demo_viome_test = pd.read_csv('/Users/sudheerb/Documents/Academics /1st Semester/ML/Project/fa-24-tamu-csce-633-600-machine-learning/demo_viome_test.csv')
demo_viome_test = preprocess_viome(demo_viome_test)
print(demo_viome_test.shape)
test_data = pd.merge(processed_test_data, demo_viome_test, on=['Subject ID'])
print(test_data.shape)

# Normalize all numerical data
numeric_cols = test_data.select_dtypes(include=[np.number]).columns.difference(['Subject ID', 'Day', 'Lunch Calories'])
test_data[numeric_cols] = scaler.fit_transform(test_data[numeric_cols])

# Perform PCA
X_test_pca = pca.fit_transform(test_data[numeric_cols])

# Apply PCA with the selected number of components
X_test_reduced = pca.fit_transform(test_data[numeric_cols])

# Create a DataFrame with reduced features, preserving the component names
columns = [f"PC{i+1}" for i in range(n_components)]
reduced_df = pd.DataFrame(X_test_reduced, columns=columns, index=test_data.index)

# Select non-numerical and excluded columns from the original data
excluded_cols = test_data.drop(columns=numeric_cols)

# Combine reduced features with the excluded columns, keeping column names intact
final_test_data = pd.concat([excluded_cols, reduced_df], axis=1)

print(final_test_data.shape)

MAGE for Subject 4 on 2: 28.215937303087596
MAGE for Subject 4 on 3: 18.267742197599254
MAGE for Subject 4 on 6: 16.25171111111111
MAGE for Subject 4 on 7: 20.803852468493464
MAGE for Subject 4 on 8: 12.183560493827162
MAGE for Subject 4 on 9: 13.90724489795918
MAGE for Subject 4 on 10: 20.283314496768238
MAGE for Subject 21 on 2: 18.913910309880084
MAGE for Subject 21 on 3: 8.799322121179815
MAGE for Subject 21 on 4: 32.783081354990976
MAGE for Subject 21 on 5: 10.983517284464808
MAGE for Subject 21 on 6: 14.637582951547966
MAGE for Subject 21 on 7: 32.3079605719839
MAGE for Subject 21 on 8: 14.50154033041788
MAGE for Subject 21 on 9: 16.638687352492013
MAGE for Subject 21 on 10: 27.692768290989857
MAGE for Subject 27 on 2: 12.556431076628755
MAGE for Subject 27 on 3: 5.588004231793051
MAGE for Subject 27 on 4: 34.934706372344856
MAGE for Subject 27 on 5: 8.78323198667222
MAGE for Subject 27 on 6: 21.543658978384258
MAGE for Subject 27 on 7: 5.735921231326389
MAGE for Subject 27 on 8:

In [23]:
class MultimodalTestDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.breakfast_images = torch.tensor(
            np.stack(data['Image Before Breakfast'].values), dtype=torch.float32
        ).permute(0, 3, 1, 2)
        self.lunch_images = torch.tensor(
            np.stack(data['Image Before Lunch'].values), dtype=torch.float32
        ).permute(0, 3, 1, 2)

        viome_cols = [col for col in data.columns if 'PC' in col]
        demo_viome_features = data[viome_cols].values
        self.demo_viome = torch.tensor(demo_viome_features, dtype=torch.float32)

        # demo_cols = ['Age', 'Weight', 'Height', 'A1C', 'Baseline Fasting Glucose', 
        #              'Insulin', 'Triglycerides', 'Cholesterol', 'HDL', 'Non-HDL', 
        #              'LDL', 'VLDL', 'CHO/HDL Ratio', 'HOMA-IR', 'BMI']
        # demo_viome_features = data[viome_cols + demo_cols].values
        # self.demo_viome = torch.tensor(demo_viome_features, dtype=torch.float32)
            
        def process_cgm(x):
            try:
                if isinstance(x, str):
                    clean_str = x.replace('[', '').replace(']', '').replace('\n', '').replace(' ', '')
                    if clean_str:
                        values = [float(i) for i in clean_str.split(',') if i]
                        if len(values) > 288:
                            return values[:288]
                        elif len(values) < 288:
                            return values + [0.0] * (288 - len(values))
                        return values
                return [0.0] * 288
            except:
                return [0.0] * 288
        
        processed_cgm = []
        for x in data['CGM Data'].values:
            cgm_values = process_cgm(x)
            processed_cgm.append(cgm_values)
            
        self.cgm_data = torch.tensor(processed_cgm, dtype=torch.float32)
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'breakfast_img': self.breakfast_images[idx],
            'lunch_img': self.lunch_images[idx],
            'cgm': self.cgm_data[idx],
            'demo_viome': self.demo_viome[idx]
        }

# Example usage for test data
test_dataset = MultimodalTestDataset(final_test_data)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Iterating through the test DataLoader
for batch in test_dataloader:
    print(batch['breakfast_img'].shape)
    print(batch['lunch_img'].shape)
    print(batch['demo_viome'].shape)
    print
    break  # Just show one batch for brevity

torch.Size([4, 3, 64, 64])
torch.Size([4, 3, 64, 64])
torch.Size([4, 21])


In [24]:
def evaluate_model(model, data_loader, criterion):
    """
    Evaluates the model on a given dataset.
    
    Args:
        model (nn.Module): The trained multimodal model.
        data_loader (DataLoader): DataLoader containing the test/validation dataset.
        criterion (nn.Module): Loss function used for evaluation.
        
    Returns:
        float: The average loss over the dataset.
        list: Predicted values for the dataset.
        list: Ground truth values for the dataset.
    """
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():  # Disable gradient computation for evaluation
        for batch in data_loader:
            # Extract input features and labels from the batch
            breakfast_img = batch['breakfast_img']
            lunch_img = batch['lunch_img']
            cgm = batch['cgm']
            demo_viome = batch['demo_viome']
            
            # Forward pass through the model
            outputs = model(breakfast_img, lunch_img, cgm, demo_viome)
            print(outputs)
            # Compute the loss
            # loss = criterion(outputs, labels)
            # total_loss += loss.item()
            if outputs.dim() == 0:  # Scalar tensor
                predictions.append(outputs.item())  # Convert scalar to Python float and append
            elif outputs.dim() == 1:  # 1D tensor
                predictions.extend(outputs.cpu().numpy().tolist()) 
            # Store predictions and true labels for later analysis
            # predictions.extend(outputs.cpu().numpy())
            # true_labels.extend(labels.cpu().numpy())
    
    # Calculate the average loss
    # average_loss = total_loss / len(data_loader)
    return predictions

In [25]:
test_dataloader

In [26]:
test_predictions = evaluate_model(model, test_dataloader, criterion)

tensor([454.8584, 475.6868, 453.1521, 459.2869])
tensor([447.5351, 444.2833, 437.2796, 439.4741])
tensor([442.6361, 414.3090, 436.8123, 437.5099])
tensor([440.3338, 448.4197, 441.9756, 445.6468])
tensor([433.3279, 437.5970, 439.1722, 435.9730])
tensor([455.4630, 458.8407, 447.0456, 461.7048])
tensor([445.0646, 442.7906, 420.3966, 464.6165])
tensor([435.8150, 434.9060, 442.4807, 420.7024])
tensor([441.1962, 432.1398, 434.7408, 430.9571])
tensor([444.2370, 431.2426, 448.7660, 458.1082])
tensor([444.2049, 432.1078, 447.4348, 441.4818])
tensor([432.2429, 432.1892, 446.0329, 450.2327])
tensor([467.9865, 455.6285, 439.4086, 424.6168])
tensor([407.2012, 461.4211, 432.2429, 433.8542])
tensor([449.5145, 439.6010, 434.3907, 456.8554])
tensor([443.0893, 438.0998, 448.4238, 431.7719])
tensor([427.3395, 454.2574, 435.3659, 440.9859])
tensor([456.9681, 447.2923, 441.9120, 433.7027])
tensor(421.0296)


In [27]:
df = pd.DataFrame({
    'row_id': range(len(test_predictions)),
    'label': test_predictions
})

# Save the DataFrame to a CSV file
output_file = "test_predictions.csv"
df.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")

Predictions saved to test_predictions.csv
